# C_tha

In [34]:
import rasterio
from rasterio.windows import Window
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
import numpy as np
import math


In [32]:
# File paths
raster_path = '../data/carbon/morr_C_tha_hires.tif'
boundary_path = '../data/boundaries/Morridge-Hill (Warslow Moors)/Morridge-Hill (Warslow Moors).shp'
output_raster = '../data/clipped_carbon.tif'

In [33]:
# Load and simplify boundary
boundary = gpd.read_file(boundary_path)
simplified_boundary = boundary.simplify(tolerance=10)  # Simplify geometry
merged_boundary = simplified_boundary.unary_union  # Merge into one geometry
boundary_geom = [merged_boundary]  # Convert to list for rasterio masking

/tmp/ipykernel_13971/1744124809.py:4: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  merged_boundary = simplified_boundary.unary_union  # Merge into one geometry


In [38]:
def process_large_raster_in_chunks(raster_path, boundary_geom, output_raster, chunk_size=2048):
    with rasterio.open(raster_path) as src:
        # Create profile for the output raster
        profile = src.profile
        profile.update(driver='GTiff', compress='lzw', tiled=True, nodata=0)

        # Open the output file
        with rasterio.open(output_raster, 'w', **profile) as dst:
            # Loop through the raster in chunks
            for row_off in range(0, src.height, chunk_size):
                for col_off in range(0, src.width, chunk_size):
                    # Define the window
                    window = Window(col_off, row_off,
                                    min(chunk_size, src.width - col_off),
                                    min(chunk_size, src.height - row_off))
                    
                    # Check if the window intersects the boundary
                    if window_intersects_boundary(window, src.transform, merged_boundary):
                        # Read the data for the window
                        data = src.read(1, window=window)
                        window_transform = src.window_transform(window)

                        # Create a mask for the chunk
                        mask = rasterio.features.geometry_mask(
                            boundary_geom,
                            transform=window_transform,
                            invert=True,
                            out_shape=data.shape,
                            all_touched=True,
                        )
                        
                        # Apply the mask to the data
                        masked_data = np.where(mask, data, 0)  # Set nodata as 0

                        # Write the masked data to the output file
                        dst.write(masked_data, 1, window=window)

In [39]:
# Run the processing function
process_large_raster_in_chunks(raster_path, boundary_geom, output_raster)
print(f"Clipping complete. Output saved to '{output_raster}'.")

Clipping complete. Output saved to '../data/clipped_carbon.tif'.


In [43]:
import rasterio
import numpy as np

# File path to the clipped raster
clipped_raster = '../data/clipped_carbon.tif'

# Statistics placeholder
total_sum = 0
total_count = 0
max_value = 0

chunk_size = 2048  # Adjust based on memory availability

# Process raster in chunks
with rasterio.open(clipped_raster) as src:
    for row_off in range(0, src.height, chunk_size):
        for col_off in range(0, src.width, chunk_size):
            # Define window
            window = rasterio.windows.Window(
                col_off, row_off,
                min(chunk_size, src.width - col_off),
                min(chunk_size, src.height - row_off)
            )

            # Read the data in the window
            data = src.read(1, window=window)

            # Mask nodata values
            nodata = src.nodata
            data = np.ma.masked_equal(data, nodata)

            # Update statistics
            total_sum += np.sum(data)
            total_count += np.count_nonzero(~data.mask)
            max_value = max(max_value, np.max(data))

# Compute mean
mean_value = total_sum / total_count

print("Raster Statistics:")
print(f"Mean Carbon Stock: {mean_value} t/ha")
print(f"Max Carbon Stock: {max_value} t/ha")
print(f"Total Carbon Stock: {total_sum} t")


Raster Statistics:
Mean Carbon Stock: -- t/ha
Max Carbon Stock: 504 t/ha
Total Carbon Stock: -- t
